## Task 2 – Text Summarization using LSTM with Attention

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Text_summarization_dataset.xlsx to Text_summarization_dataset (2).xlsx


### Load and Prepare Dataset

This block:
- Loads the Excel file into a DataFrame
- Cleans the column headers (first row was header)
- Renames columns to `text` and `summary`
- Drops unused rows and resets the index
- Shows the first few samples


In [ ]:
import pandas as pd

dataset = pd.read_excel("Text_summarization_dataset.xlsx")

dataset.columns = dataset.iloc[0]
dataset = dataset.drop(0)


dataset = dataset.rename(columns={"summary": "summary", "text": "text"})


dataset = dataset[["text", "summary"]]
dataset.reset_index(drop=True, inplace=True)
dataset.head()

,text,summary
0,يكون سعر الفاكهة والخضراوات في موسم إنباتها أق...,تناول الفاكهة والخضراوات في موسمها. تعرف على أ...
1,الأطعمة الصحية ليست باهظة الثمن بالضرورة، بل ف...,فضل خيارات الأطعمة الأرخص ثمنا. تباطأ في استهل...
2,استفد من حديقتك المنزلية أو أصيص الزرع الصغير ...,ازرع كل ما يمكنك من خضراوات وفاكهة. اطه بنفسك ...
3,تساعدك الخطط المسبقة في كل نواحي حياتك على وضع...,خطط مسبقا لوجباتك الرئيسية لمدة أسبوع. التزم ب...
4,نظرا لأن السبب الرئيسي لضغط العين هو أن ثقافة ...,قلل وقت التعرض للشاشات. اذهب إلى الطبيب.


### Tokenization with spaCy

Loads a multilingual spaCy model for tokenizing Arabic (or multilingual) text.  
The `tokenize_ar` function splits text into a list of tokens.


In [ ]:
import spacy
nlp = spacy.load("xx_ent_wiki_sm")

def tokenize_ar(text):
    return [tok.text for tok in nlp(text)]

### Tokenization Sample

Prints the original text and the list of tokens produced by the `tokenize_ar` function for a single sample.


In [ ]:
sample = dataset['text'][0]
print("Original:", sample)
print("Tokens:", tokenize_ar(sample))

Original: يكون سعر الفاكهة والخضراوات في موسم إنباتها أقل من غيره من المواسم، وستلجأ محلات الخضروات إلى عرض الفاكهة بأسعار مناسبة في موسمها بسبب توفر المنتجات وزيادة الطلب عليها خلال تلك الفترات. لا يقتصر الأمر على السعر الأقل، بل سيكون طعامك من الخضراوات والفاكهة أشهى وألذ عند تناوله في موسمه.   في فصل الخريف: التفاح والتين والبنجر والكمثرى والقرنبيط والكرنب واليقطين في فصل الشتاء: (الخضراوات) الملفوف والفاصوليا والبازلاء والبصل (الفواكه)  البطاطا الحلوة والأفوكادو والبرتقال والتفاح والموز واليوسفى والرمان والعنب. في فصل الربيع: السبانخ والجزر والكوسة والبصل الأخضر  والطماطم والخضراوات الورقية والفراولة والمشمش. في فصل الصيف: الصيف هو فصل البطيخ، وكذلك الأمر بالنسبة للذرة والتوت. سوف تلاحظ توفر هذه الخضراوات والفواكهة المهمة على مدار العام كله بشكل أو آخر، ولكنها تكون بأرخص أسعارها في فصل الصيف، وهو ما يمكنك من شراء كميات كبيرة منها وتخزينها أو تجميدها في المجمد (الفريزر). الأفضل لصحتك البدنية والمالية على حد سواء هو تناول الفواكه والخضراوات والأطعمة الطازجة عامة، ولكن في بعض الحالات 

### Clean and Re-tokenize Text

Defines a `clean_text` function to:
- Remove punctuation
- Remove digits
- Strip whitespace

Then redefines `tokenize_ar` to include cleaning before tokenization.


In [ ]:
import re

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    return text.strip()


def tokenize_ar(text):
    text = clean_text(text)
    return [tok.text for tok in nlp(text)]

### Re-check Tokenization (Duplicate)

This appears to repeat the earlier sample preview.  
May be removed or kept to confirm tokenization after text cleaning.


In [ ]:
sample = dataset['text'][0]
print("Original:", sample)
print("Tokens:", tokenize_ar(sample))

Original: يكون سعر الفاكهة والخضراوات في موسم إنباتها أقل من غيره من المواسم، وستلجأ محلات الخضروات إلى عرض الفاكهة بأسعار مناسبة في موسمها بسبب توفر المنتجات وزيادة الطلب عليها خلال تلك الفترات. لا يقتصر الأمر على السعر الأقل، بل سيكون طعامك من الخضراوات والفاكهة أشهى وألذ عند تناوله في موسمه.   في فصل الخريف: التفاح والتين والبنجر والكمثرى والقرنبيط والكرنب واليقطين في فصل الشتاء: (الخضراوات) الملفوف والفاصوليا والبازلاء والبصل (الفواكه)  البطاطا الحلوة والأفوكادو والبرتقال والتفاح والموز واليوسفى والرمان والعنب. في فصل الربيع: السبانخ والجزر والكوسة والبصل الأخضر  والطماطم والخضراوات الورقية والفراولة والمشمش. في فصل الصيف: الصيف هو فصل البطيخ، وكذلك الأمر بالنسبة للذرة والتوت. سوف تلاحظ توفر هذه الخضراوات والفواكهة المهمة على مدار العام كله بشكل أو آخر، ولكنها تكون بأرخص أسعارها في فصل الصيف، وهو ما يمكنك من شراء كميات كبيرة منها وتخزينها أو تجميدها في المجمد (الفريزر). الأفضل لصحتك البدنية والمالية على حد سواء هو تناول الفواكه والخضراوات والأطعمة الطازجة عامة، ولكن في بعض الحالات 

### Build Vocabularies

This function:
- Builds token frequency counters for both source texts and summaries
- Filters out rare words (appearing < 2 times)
- Adds special tokens: `<pad>`, `<sos>`, `<eos>`, `<unk>`
- Assigns each token an index to create a vocabulary dictionary


In [ ]:
from collections import Counter

dataset.dropna(inplace=True)

def build_vocab(texts, min_freq=2):
    counter = Counter()
    for txt in texts:
        tokens = tokenize_ar(txt)
        counter.update(tokens)

    vocab = {
        "<pad>": 0,
        "<sos>": 1,
        "<eos>": 2,
        "<unk>": 3,
    }

    idx = 4
    for word, freq in counter.items():
        if freq >= min_freq:
            vocab[word] = idx
            idx += 1

    return vocab


text_vocab = build_vocab(dataset["text"])
summary_vocab = build_vocab(dataset["summary"])

print("عدد كلمات النص:", len(text_vocab))
print("عدد كلمات التلخيص:", len(summary_vocab))


عدد كلمات النص: 132722
عدد كلمات التلخيص: 33874


### Numericalize Text

This function:
- Converts tokens to integers using the vocabulary
- Optionally adds `<sos>` and `<eos>` tokens for decoder input
- Pads/truncates the sequence to a fixed maximum length


In [ ]:
def numericalize(text, vocab, add_sos_eos=False, max_len=100):
    tokens = tokenize_ar(text)
    if add_sos_eos:
        tokens = ["<sos>"] + tokens + ["<eos>"]

    ids = [vocab.get(tok, vocab["<unk>"]) for tok in tokens]


    if len(ids) < max_len:
        ids += [vocab["<pad>"]] * (max_len - len(ids))
    else:
        ids = ids[:max_len]

    return ids


### Vectorize the Dataset

This cell applies the `numericalize` function to the entire dataset:
- Source texts are converted to fixed-length vectors of 150 tokens
- Summaries are encoded with `<sos>` and `<eos>` for a max length of 50 tokens


In [ ]:
MAX_TEXT_LEN = 150
MAX_SUMMARY_LEN = 50

text_seqs = [numericalize(txt, text_vocab, add_sos_eos=False, max_len=MAX_TEXT_LEN)
             for txt in dataset["text"]]

summary_seqs = [numericalize(txt, summary_vocab, add_sos_eos=True, max_len=MAX_SUMMARY_LEN)
                for txt in dataset["summary"]]


### Create PyTorch Dataset Class

Defines a custom `Dataset` class compatible with PyTorch `DataLoader`.  
Each example includes:
- `input`: vectorized source sentence
- `target`: vectorized summary


In [ ]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class SummarizationDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            'input': torch.tensor(self.inputs[idx], dtype=torch.long),
            'target': torch.tensor(self.targets[idx], dtype=torch.long)
        }


### Split into Train and Validation Sets

Splits the numericalized dataset into 80% training and 20% validation.  
Creates corresponding `SummarizationDataset` objects.


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_summaries, val_summaries = train_test_split(
    text_seqs, summary_seqs, test_size=0.2, random_state=42)

train_dataset = SummarizationDataset(train_texts, train_summaries)
val_dataset = SummarizationDataset(val_texts, val_summaries)


### Create DataLoaders

Wraps datasets into PyTorch `DataLoader` objects with:
- Batch size: 32
- Shuffling enabled for training set


In [ ]:
BATCH_SIZE = 32

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)


### Encoder Module

The Encoder:
- Embeds input token indices
- Processes sequences using a single-layer LSTM
- Outputs the hidden states and final memory states to pass to the decoder


In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=1):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, bidirectional=False)

    def forward(self, x):
        embedded = self.embedding(x)
        outputs, (hidden, cell) = self.lstm(embedded)
        return outputs, hidden, cell


### Attention Mechanism

This module computes attention scores by:
- Comparing decoder hidden state to encoder outputs
- Producing a distribution over the input sequence to weigh context


In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        src_len = encoder_outputs.shape[1]

        hidden = hidden[-1].unsqueeze(1).repeat(1, src_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)

        return torch.softmax(attention, dim=1)


### Decoder with Attention

The decoder:
- Embeds the input summary token
- Applies attention over the encoder outputs to create a context vector
- Concatenates embedding + context to generate a prediction for the next word
- Repeats this step for each time step during training


In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, attention, num_layers=1):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size + hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.attention = attention

    def forward(self, input, hidden, cell, encoder_outputs):
        input = input.unsqueeze(1)  # [B] -> [B, 1]
        embedded = self.embedding(input)  # [B, 1, E]

        attn_weights = self.attention(hidden, encoder_outputs)  # [B, src_len]
        attn_weights = attn_weights.unsqueeze(1)  # [B, 1, src_len]

        context = torch.bmm(attn_weights, encoder_outputs)  # [B, 1, H]
        rnn_input = torch.cat((embedded, context), dim=2)  # [B, 1, E+H]

        outputs, (hidden, cell) = self.lstm(rnn_input, (hidden, cell))
        predictions = self.fc(outputs.squeeze(1))  # [B, vocab]

        return predictions, hidden, cell, attn_weights.squeeze(1)


### Seq2Seq Model Wrapper

This module combines the encoder and decoder:
- Uses teacher forcing to mix true labels with predictions
- Iteratively decodes summary tokens while attending to the encoder outputs
- Stores all predictions in a tensor of shape [batch, trg_len, vocab_size]


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.fc.out_features

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        encoder_outputs, hidden, cell = self.encoder(src)

        input = trg[:, 0]  # <sos>

        for t in range(1, trg_len):
            output, hidden, cell, _ = self.decoder(input, hidden, cell, encoder_outputs)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input = trg[:, t] if random.random() < teacher_forcing_ratio else top1

        return outputs


### Initialize Model and Optimizer

- Defines key hyperparameters (embedding, hidden size)
- Instantiates encoder, decoder, and attention modules
- Moves everything to GPU if available
- Uses `CrossEntropyLoss` (ignoring `<pad>` tokens)
- Adam optimizer with learning rate 0.001


In [ ]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# المعلمات
INPUT_DIM = len(text_vocab)
OUTPUT_DIM = len(summary_vocab)
EMBED_SIZE = 256
HIDDEN_SIZE = 512

# بناء النموذج
attn = Attention(HIDDEN_SIZE)
encoder = Encoder(INPUT_DIM, EMBED_SIZE, HIDDEN_SIZE).to(device)
decoder = Decoder(OUTPUT_DIM, EMBED_SIZE, HIDDEN_SIZE, attn).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)

# loss و optimizer
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001)


### Training Function

This function:
- Trains the model for one epoch
- Applies teacher forcing in the forward pass
- Clips gradients to prevent exploding
- Returns the average training loss per epoch


In [ ]:
def train(model, iterator, optimizer, criterion, clip=1):
    model.train()
    epoch_loss = 0

    for batch in iterator:
        src = batch['input'].to(device)
        trg = batch['target'].to(device)

        optimizer.zero_grad()
        output = model(src, trg)  # [batch, trg_len, vocab_size]

        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)

        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


### Evaluation Function

Evaluates model performance on validation data:
- Disables teacher forcing
- Computes loss across entire validation set
- Returns average loss


In [ ]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for batch in iterator:
            src = batch['input'].to(device)
            trg = batch['target'].to(device)

            output = model(src, trg, 0)  # No teacher forcing

            output_dim = output.shape[-1]
            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


### Training Loop

Trains the model for 5 epochs:
- Prints train and validation loss after each epoch
- Helps monitor model learning and overfitting


In [ ]:
import random

N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss = evaluate(model, val_loader, criterion)

    print(f'Epoch {epoch+1}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {val_loss:.3f}')


KeyboardInterrupt: 

### Decode Token IDs to Text

Utility function to:
- Convert a list of token IDs back to readable Arabic text
- Skip padding and special tokens


In [ ]:
def ids_to_text(ids, vocab):
    inv_vocab = {v: k for k, v in vocab.items()}
    words = [inv_vocab.get(i, "<unk>") for i in ids]
    return ' '.join([w for w in words if w not in ["<pad>", "<sos>", "<eos>"]])


### Generate Summary for New Input

Uses the trained model to:
- Encode the input text
- Generate summary tokens step-by-step (greedy decoding)
- Stop when `<eos>` token is predicted or max length is reached


In [ ]:
def summarize_text(model, input_text, text_vocab, summary_vocab, max_len=MAX_SUMMARY_LEN):
    model.eval()
    tokens = numericalize(input_text, text_vocab, max_len=MAX_TEXT_LEN)
    src_tensor = torch.tensor(tokens).unsqueeze(0).to(device)

    with torch.no_grad():
        encoder_outputs, hidden, cell = model.encoder(src_tensor)

    outputs = [summary_vocab["<sos>"]]
    for _ in range(max_len):
        prev_word = torch.tensor([outputs[-1]]).to(device)
        with torch.no_grad():
            output, hidden, cell, _ = model.decoder(prev_word, hidden, cell, encoder_outputs)
        pred_token = output.argmax(1).item()
        outputs.append(pred_token)
        if pred_token == summary_vocab["<eos>"]:
            break

    return ids_to_text(outputs, summary_vocab)


In [ ]:
!pip install gradio

### Gradio Interface for Summarization

Creates a web interface for the summarization model using **Gradio**:
- Takes Arabic input in a textbox
- Returns a generated summary from the trained model
- Uses the `summarize_text()` function behind the scenes


In [ ]:
import gradio as gr

def gradio_summarize(input_text):
    summary = summarize_text(model, input_text, text_vocab, summary_vocab)
    return summary

demo = gr.Interface(
    fn=gradio_summarize,
    inputs=gr.Textbox(lines=10, label="النص الأصلي"),
    outputs=gr.Textbox(lines=5, label="التلخيص الناتج"),
    title="نموذج تلخيص باستخدام LSTM + Attention",
    description="أدخل نصًا عربيًا وسيقوم النموذج بإرجاع تلخيص له"
)

demo.launch(share=True)

## Conclusion

In this task, we implemented an Arabic text summarization model using an Encoder-Decoder architecture with an attention mechanism.

Key highlights:
- The encoder processed input sequences with LSTM.
- The decoder generated summary tokens with the help of learned attention over the input.
- The model achieved reasonable results in generating concise summaries.
- An interactive Gradio interface was built to demonstrate the model in action.
